<a href="https://colab.research.google.com/github/sakurasakura1996/Pytorch-start-learning/blob/master/Dive_into_DL_Pytorch_4_1_%E6%A8%A1%E5%9E%8B%E6%9E%84%E9%80%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn

In [0]:
class MLP(nn.Module):
    def __init__(self, **kwargs):
        super(MLP,self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256)   # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)
    
    def forward(self, x):
        h = self.act(self.hidden(x))
        return self.output(h)

In [0]:
X = torch.rand(1, 784)

In [0]:
net = MLP()
print(net)
net(X)
# 这里并没有将 Module类命名为 Layer层  或者 Model模型这类的名字，是因为该类是一个可供自由组建的部件。它的子类既可以是一个层（如pytorh提供
# 的Linear类），又可以是一个模型（如这里定义的MLP类）或者是模型的一部分。下面两个例子来展示他的灵活性

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.0313,  0.1702, -0.0252,  0.0855, -0.1259, -0.0680, -0.0421,  0.0860,
         -0.1692,  0.0009]], grad_fn=<AddmmBackward>)

In [0]:
# 有很多继承自Module的类  如 Sequential类，  ModuleList，   ModuleDict
# 当模型的前向计算为简单串联各个层的计算时，Sequential类可以通过简单的方式定义模型。它可以接受一个子模块的有序字典（OrderDict)或者一系列子模块
# 作为参数来逐一添加 Module实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。下面实现一个与Sequential相同功能的MySequential类。加深理解
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args)==1 and isinstance(args[0], OrderedDict): # 如果传入的是一个OrderDict
            for key, module in args[0].items():
                self.add_module(key,module)  # add_module方法将会module添加进self._modules(一个OrderDict)
        else:
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
                
    def forward(self, input):
        # self._modules返回一个 OrderDict,保证会按照成员添加时的顺序遍历
        for module in self._modules.values():
            input = module(input)
        return input

In [0]:
net = MySequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
)
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.0575,  0.0974,  0.0011, -0.1859,  0.0505,  0.1051, -0.1154,  0.0543,
         -0.2051,  0.1766]], grad_fn=<AddmmBackward>)

In [0]:
# ModuleList类
# ModuleList接收一个子模块的列表作为输入，然后也可以类似List那样进行append和extend操作
import torch.nn as nn
net = nn.ModuleList([nn.Linear(784, 256),nn.ReLU()])
net.append(nn.Linear(256, 10))
print(net[-1])
print(net)
# net(X)  这里 不能直接调用net(x)因为 ModuleList 类似于list,模型中未自动实现forward函数

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [0]:
for model in net:
    X = model(X)
print(X)    # 这种表示法可以实现简单的前向计算

tensor([[-0.1012,  0.1225, -0.1777,  0.0598, -0.0754,  0.1292,  0.0904,  0.0635,
         -0.0614,  0.1253]], grad_fn=<AddmmBackward>)


In [0]:
# ModuleDict类 接受一个子模块的字典作为输入，然后也可以类似字典那样进行添加访问操作
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10)
print(net['linear'])
print(net.output)
print(net)


Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


In [0]:
# 构造复杂的模型
# 虽然上面介绍的这些类可以使模型构造更加简单，且不需要定义forward函数但直接继承Module类可以极大地扩展模型构造的灵活性。下面我们构造一个稍微
# 复杂的网络 FancyMLP。这个网络中，我们通过get_constant函数创建训练中不被迭代的参数，即常数参数。在前向计算中，除了使用创建的常熟参数外，我们
# 还使用Tensor的函数和Python的控制流，并多次调用相同的层
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP,self).__init__(**kwargs)
        
        self.rand_weight = torch.rand((20,20),requires_grad = False)   # 不可训练参数（常数参数）
        self.linear = nn.Linear(20, 20)
    
    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常数参数,以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x,self.rand_weight.data)+1)
        # 复用全连接层,等价于 两个全连接层 共享参数
        x = self.linear(x)
        # 控制流，我们这里需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() <0.8:
            x *= 10
        return x.sum()

In [0]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-11.0381, grad_fn=<SumBackward0>)